In [3]:
import torch
import psutil
import itertools
import numpy as np
import utils as ut
import pandas as pd
from time import time
from torch import optim
from pathlib import Path
from skimage import io as skio
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score
device = torch.device("cpu")

result_dir = "results/10m/"
data_dir = "~/postdoc/rumex/data_patches_for_training/10m/256/test/"

# test dataset and data loader
dste = ut.RumexDataset(data_dir, train_flag=False)
dlte = ut.test_loader(dste, 64)
# %% make predictions
nte = len(dste)
n1te = dste.rumex.targets.count(1)
n0te = dste.rumex.targets.count(0)
print(nte, n1te, n0te)

552 84 468


In [8]:
model_name = "densenet"
ckpt_dir = "logs/triton/"+model_name+"/"
ckpt_dict = torch.load(ckpt_dir+"best.pt")

# only useful for 10m dataset
# temp = ckpt_dict['predictions']
# pred_valid = {}
# pred_valid["yva"] = temp['yva'].detach().cpu().numpy()
# pred_valid["yhatva"] = temp['yhatva'].detach().cpu().numpy()
# pred_valid["score1va"] = temp['scoreva'].detach().cpu().numpy()
# pred_valid["lossva"] = temp['lossva'].detach().cpu().numpy()
# df_valid = pd.DataFrame.from_dict(pred_valid)
# df_valid.to_csv(result_dir + model_name+"_valid_predictions.csv")


model = ut.RumexNet(model_name)
model.load_state_dict(ckpt_dict["state_dict"])
model = model.to(device)

yte = []
score1te = []
score0te = []
losste = []
yhatte = []
fnamete = []
loss_fn = torch.nn.CrossEntropyLoss(reduction="none")
for xteb, yteb, idb, fnameb in dlte:
    xteb = xteb.to(device)
    yteb = yteb.to(device)

    # Forward pass
    score_te_b = model(xteb)  # logits
    loss_te_b = loss_fn(score_te_b, yteb)
    _, yhat_te_b = torch.max(score_te_b, 1)

    # book keeping at batch level
    yte.append(yteb.detach().numpy().tolist())

    score0te.append(score_te_b.detach().numpy()[:, 0].tolist())
    score1te.append(score_te_b.detach().numpy()[:, 1].tolist())

    losste.append(loss_te_b.detach().numpy().tolist())
    yhatte.append(yhat_te_b.detach().numpy().tolist())
    fnamete.append(list(fnameb))

yte = list(pd.core.common.flatten(yte))
score0te = list(pd.core.common.flatten(score0te))
score1te = list(pd.core.common.flatten(score1te))
losste = list(pd.core.common.flatten(losste))
yhatte = list(pd.core.common.flatten(yhatte))
fnamete = list(pd.core.common.flatten(fnamete))

pred_test = {"fname": fnamete,
             "yte": yte,
             "yhatte": yhatte,
             "scorete0": score0te,
             "scorete1": score1te,
             "losste": losste}

df_test = pd.DataFrame.from_dict(pred_test)
# df_test.to_csv(result_dir + model_name+"_test_predictions.csv")

acc = accuracy_score(df_test.yte, df_test.yhatte)
f1 = f1_score(df_test.yte, df_test.yhatte)
pre = precision_score(df_test.yte, df_test.yhatte)
recall = recall_score(df_test.yte, df_test.yhatte)
auc = roc_auc_score(df_test.yte, df_test.scorete1)

print(f"{n0te/nte:.4f}, {acc:.4f}, {f1:.4f}, {pre:.4f}, {recall:.4f}, {auc:.4f}")

0.8478, 0.9167, 0.6462, 0.9130, 0.5000, 0.9390


In [ ]:
nte, n1te, n0te

In [ ]:
n0te/nte